# LIBRARIES & FUNCTIONS

In [ ]:
# LIBRARIES #

import numpy as np

from scipy import optimize

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect, Cross
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk 

import pydicom

import json

from Gafchromic import Radiochromic_RB
from Gafchromic_multichannel import Radiochromic_multilinear

import time


output_notebook()

In [ ]:
# FONCTIONS #



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleDataArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor]\
                        .reshape((newsizey, subfactor, newsizex, subfactor)).mean(3).mean(1)
    
    return newarray



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleRGBArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor,:]\
                        .reshape((newsizey, subfactor, newsizex, subfactor, 3)).mean(3).mean(1)
    
    return newarray



# reading dose matrix in axial plane at isocenter 
#    from dose and plan dicom files : #
# @params:
#  doseFilename : filename of the dicom file containing the dose matrix
#  planFilename : filename of the dicom file containing the RT plan
#  verbose : writes the print output ?
def readDoseMatrix_AxialPlane_Isocenter(doseFilename, planFilename, verbose=False):

    ds_dose = pydicom.read_file(doseFilename)
    ds_plan = pydicom.read_file(planFilename)
    
    if verbose: print('Dose files read!')

    
    # Isocenter Coordinates from plan file:
    iso_x=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[0]
    iso_y=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[1]
    iso_z=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[2]

    if verbose: print ('  > isocenter coordinates = (', iso_x, ";", iso_y, ";", iso_z, ")")


    # Dose image coordinates from dose file:
    position_image=list(ds_dose['0020','0032'].value) # +x left, +y post, +z head   
    position_image_x=list(ds_dose['0020','0032'].value)[0]
    position_image_y=list(ds_dose['0020','0032'].value)[1]
    position_image_z=list(ds_dose['0020','0032'].value)[2]

    if verbose: print ('  > Dose image position = (', position_image_x, ";", position_image_y, ";", position_image_z, ")")


    # Dose matrix in axial plane at isocenter:
    dim_dose= ds_dose.pixel_array.shape # disposition y, z, x?
    
    if verbose: print ('  > dose matrix size = ', dim_dose)

    pixel_spacing_x=(ds_dose['0028','0030'].value[0]) #definition de la résolution de la matrice de dose
    pixel_spacing_z=(ds_dose['0028','0030'].value[1])
    pixel_spacing_y=(ds_dose['3004','000C'].value[1])
    
    if verbose: print ('  > pixel spacing = (', pixel_spacing_x, ";", pixel_spacing_y, ";", pixel_spacing_z, ")")

    coord_iso_dose = (round((iso_x-position_image_x)/pixel_spacing_x),
                      round((iso_y-position_image_y)/pixel_spacing_y),
                      round((iso_z-position_image_z)/pixel_spacing_z))

    if verbose: print ('  > isocenter coodinates in dose matrix =', coord_iso_dose)

    doseimg = ds_dose.pixel_array[:,coord_iso_dose[1],:] * ds_dose.DoseGridScaling * 100 #en cGy

    if verbose: print ('  > maximum dose :', np.amax(doseimg))

    return (doseimg, dim_dose[0], dim_dose[2], pixel_spacing_x, pixel_spacing_z)




# Creating a calibration model for a line. It results parameters of the fit
#   for the RB vs GB curve, and the parameters of the fit for the dose vs RB&GB
#   curve.
#   
#  <!> funcDose et funcRBGB must be defined first !
#
# @params:
#  arrayrb : image matrix R/B
#  arraygb : image matrix G/B
#  arraydose : dose matrix 
#  column : column to be used
#  beginLine : line where to begin process
#  endLine : line where to end process
#  dispParameters : prints parameters found
#  dispGraphs : displays graphs
def createLineCalModel(arrayrb, arraygb, arraydose, column, beginLine, endLine, w=0,
                       dispParameters=False, dispGraphs=False):
    
    # crops the matrices:
    if w==0:
        rb = array_rb[beginLine:endLine, column]
        gb = array_gb[beginLine:endLine, column]
        dose = arraydose[beginLine:endLine, column]
    else:
        rb = np.mean(array_rb[beginLine:endLine, column-w:column+w], axis=1)
        gb = np.mean(array_gb[beginLine:endLine, column-w:column+w], axis=1)
        dose = np.mean(arraydose[beginLine:endLine, column-w:column+w], axis=1)
    
    # RB vs GB fitting:
    paramsRBGB, params_covRBGB = optimize.curve_fit(funcRBGB, rb, gb)

    # Dose vs RB&GB fitting:
    nbLines = rb.shape[0]
    rbgb = np.zeros((nbLines,2))
    rbgb[:,0] = rb
    rbgb[:,1] = gb

    paramsDose, params_covDose = optimize.curve_fit(funcDose, rbgb, dose)

    
    # Display parameters models:
    if dispParameters:
        print('Parameters RB vs GB:')
        for i in range(len(paramsRBGB)):
            print('   - parameter', i, ':', paramsRBGB[i])
        print()
        print('Parameters Dose vs RB & GB:')
        for i in range(len(paramsDose)):
            print('   - parameter', i, ':', paramsDose[i])

    # Display graph results
    if dispGraphs:
        
        # Graph RB vs GB: 
        minrb = np.min(rb)
        maxrb = np.max(rb)
        xRBGB = np.arange(minrb, maxrb, (maxrb-minrb)/50)

        p1 = figure(plot_width=600, plot_height=600, title="RB vs GB", 
                    toolbar_location="above")

        p1.circle(rb, gb, size=3, color='cornflowerblue', alpha=0.3, 
                  legend_label="column = "+str(column)) 

        p1.line(xRBGB, funcRBGB(xRBGB, paramsRBGB[0], paramsRBGB[1], paramsRBGB[2], paramsRBGB[3]), 
                line_color="black", line_width=1, line_dash='dotted')

        p1.xaxis.axis_label = "R/B values" 
        p1.yaxis.axis_label = "G/B values"
        p1.legend.location = 'top_left' 

        # Image view: 
        p2 = figure(plot_width=300, plot_height=int(300*arrayrb.shape[0]/arrayrb.shape[1]), 
                   title='', toolbar_location="above")

        p2.image(image=[arrayrb], x=0, y=0, dw=arrayrb.shape[1], dh=arrayrb.shape[0], palette="Plasma256")
        p2.line((column, column), (beginLine, endLine), line_alpha=0.7, line_color="white")
        if w>0: 
            p2.rect(x=column, y=int(beginLine+(endLine-beginLine)/2), width=2*w+1, height=endLine-beginLine, 
                    fill_color='white', fill_alpha=0.3, line_alpha=0.3, line_color="white")
        

        # RB & GB vs dose:
        p3 = figure(plot_width=600, plot_height=600, title="RB & GB vs dose", toolbar_location="above")
        p3.circle(rb, dose, size=3, color='darkslategrey', alpha=0.3, legend_label="R/B") 
        p3.circle(gb, dose, size=3, color='gold', alpha=0.3, legend_label="G/B") 

        # Plots the results of calibration on this line ()
        p4 = figure(plot_width=450, plot_height=450, title="Dose calculated vs real", toolbar_location="above")
        p4.line(dose, funcDose(rbgb, paramsDose[0], paramsDose[1], paramsDose[2], paramsDose[3], paramsDose[4]), 
                line_width=2, line_color='firebrick')
        p4.line(dose, dose, line_width=1, line_color='black', line_dash='dotted')

        p5 = figure(plot_width=450, plot_height=450, title="Dose calculation Error in %", toolbar_location="above")
        p5.line(dose,(dose-funcDose(rbgb, paramsDose[0], paramsDose[1], paramsDose[2], paramsDose[3], \
                                    paramsDose[4]))/dose*100, line_width=2, line_color='firebrick')
        p5.line(dose,2 , line_width=1, line_color='black', line_dash='dotted')
        p5.line(dose,-2 , line_width=1, line_color='black', line_dash='dotted')

        # grid = gridplot([p1, p2, p3], ncols=2)
        grid1 = gridplot([[p1, p2], [p3, None]])
        grid2 = gridplot([[p4,p5]])

        show(grid1)
        show(grid2)

    # Returns parameters:
    return [paramsRBGB, paramsDose]





# Finds which model from the list of models corresponds best to the rb & gb values
#   
# @params:
#  rb : array of column R/B
#  gb : array of column G/B
#  modelsArr : list of parameters of the models of RBGB curves 
#  dispGraphs : displays graphs if true
def findBestModel(rb, gb, modelsArr, dispGraph=False):
    
    # calculates mean error between model and curve for all models
    errorValues = []
    for i in range(len(modelsArr)):
        mod = modelsArr[i]
        err = np.mean((gb-funcRBGB(rb, mod[0], mod[1], mod[2], mod[3]))*(gb-funcRBGB(rb, mod[0], mod[1], mod[2], mod[3])))
        errorValues.append(err)
    
    errormin = np.amin(errorValues)
    bestmodel = errorValues.index(errormin)
    
    if dispGraph:
        print("  >>> Best Model:", bestmodel)
        print("      Associated Error:", errormin)
        
        p1 = figure(plot_width=600, plot_height=600, title="RB vs GB", 
                    toolbar_location="above")

        p1.circle(rb, gb, size=3, color='cornflowerblue', alpha=0.3, 
                  legend_label="column = "+str(column)) 

        minrb = np.min(rb)
        maxrb = np.max(rb)
        xRBGB = np.arange(minrb, maxrb, (maxrb-minrb)/50)
        mod = modelsArr[bestmodel]
        p1.line(xRBGB, funcRBGB(xRBGB, mod[0], mod[1], mod[2], mod[3]), 
                line_color="black", line_width=1, line_dash='dotted')

        p1.xaxis.axis_label = "R/B values" 
        p1.yaxis.axis_label = "G/B values"
        p1.legend.location = 'top_left' 
        
        show(p1)
    
    
    return [bestmodel, errormin]




# Converts an image to dose using RBGB models
#   
# @params:
#  arr : image array
#  models : dictionary of RBGB model
def convertToDose(arr, models):
    
    rb = arr[:,:,0]/arr[:,:,2]
    gb = arr[:,:,1]/arr[:,:,2]
    
    doseImg = np.zeros(rb.shape)
    
    rbgbmodels = models["RBGBcurve"]
    for i in range(rb.shape[1]):
        [index, err] = findBestModel(rb[:,i], gb[:,i], rbgbmodels, dispGraph=False)
        
        rbgb = np.zeros((rb.shape[0],2))
        rbgb[:,0] = rb[:,i]
        rbgb[:,1] = gb[:,i]
        mod = models["DoseCurve"][index]
        doseImg[:,i] = funcDose(rbgb, mod[0], mod[1], mod[2], mod[3], mod[4])
    
    return doseImg




# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)

    
    
    
# Applies streak correction to RGB image
#   
# @params:
#  array: image array on which to perform streak correction.
def applyStreakCorrection(array):
        
        # First pixels sum and normalization:
        firstPixAvg_r = np.sum(array[0:10,:,0] ,axis=0)
        firstPixAvg_g = np.sum(array[0:10,:,1] ,axis=0)
        firstPixAvg_b = np.sum(array[0:10,:,2],axis=0)

        firstPixAvg_r = firstPixAvg_r / np.mean(firstPixAvg_r)
        firstPixAvg_g = firstPixAvg_g / np.mean(firstPixAvg_g)
        firstPixAvg_b = firstPixAvg_b / np.mean(firstPixAvg_b)


        # Last pixels sum and normalization:
        lastPixAvg_r = np.sum(array[-9:,:,0] ,axis=0)
        lastPixAvg_g = np.sum(array[-9:,:,1] ,axis=0)
        lastPixAvg_b = np.sum(array[-9:,:,2],axis=0)

        lastPixAvg_r = lastPixAvg_r / np.mean(lastPixAvg_r)
        lastPixAvg_g = lastPixAvg_g / np.mean(lastPixAvg_g)
        lastPixAvg_b = lastPixAvg_b / np.mean(lastPixAvg_b)


        # Streak correction image:
        #  This correction must be done on an uncroped image
        streakCorrImage = np.zeros(array.shape)
        x = [0, array.shape[1]-1]

        for i in range(array.shape[1]):
            allpix_y = range(array.shape[0])
            y0 = [firstPixAvg_r[i], lastPixAvg_r[i]]
            streakCorrImage[:,i,0] = 1 / np.interp(allpix_y, x, y0)
    
            y1 = [firstPixAvg_g[i], lastPixAvg_g[i]]
            streakCorrImage[:,i,1] = 1 / np.interp(allpix_y, x, y1)

            y2 = [firstPixAvg_b[i], lastPixAvg_b[i]]
            streakCorrImage[:,i,2] = 1 / np.interp(allpix_y, x, y2)
    
    
        # Apply streak correction to the image:
        return array * streakCorrImage

    

# IMG READING & REGISTRATION

In [ ]:
# INPUT PARAMETERS:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers



# Variables:
m_path = 'testRBGB/'
m_nbOfFiles = 1
m_firstNb = 1
m_GafFilesName = "testRBGB_00"
# m_GafFilesName = "ppp_25"
m_fileExtension = ".tif"

m_doseFileName = "RD_1mm.dcm"
m_planFileName =  "RP_1mm.dcm"

m_splineFile = "G://Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_data.txt"

m_dimViewer = 600




In [ ]:
# READS THE FILES:


# Reads the dose img:
(doseimg, dimx, dimy, pixsizex, pixsizey) = readDoseMatrix_AxialPlane_Isocenter(
    m_path+m_doseFileName, m_path+m_planFileName, verbose=False)

calcDose = np.flip(doseimg)

# Reads and convert to dose the gafchromic film:
try:
    g = Radiochromic_RB(m_path+m_GafFilesName, m_firstNb, m_nbOfFiles)
    gafdoseimg = g.convertToDose_cubicSplineFit(m_splineFile, 800)
    size = g.getSize()
except ValueError as err:
    print('Erreur: ' + err)
    
measDose = subSampleDataArray(gafdoseimg, 10)



# shows dose images:
# p1 = figure(plot_width=m_dimViewer, plot_height=int(m_dimViewer), 
#            title='Calculated dose', toolbar_location="above")
# p1.image(image=[calcDose], x=0, y=0, dw=calcDose.shape[1], dh=calcDose.shape[1], palette="Plasma256")

# p2 = figure(plot_width=m_dimViewer, plot_height=int(m_dimViewer*measDose.shape[0]/measDose.shape[1]), 
#            title='Measured dose', toolbar_location="above")
# p2.image(image=[measDose], x=0, y=0, dw=measDose.shape[1], dh=measDose.shape[0], palette="Plasma256")

# show(p1)
# show(p2)


In [ ]:
# REGISTER IMAGES:


# Assigns images:
doseimg1 = measDose
doseimg2 = calcDose


# Initial transform:
fixedimg = sitk.Image(doseimg1.shape[1], doseimg1.shape[0], sitk.sitkFloat32)
for i in range(doseimg1.shape[1]):
    for j in range(doseimg1.shape[0]):
        fixedimg.SetPixel(i, j, float(doseimg1[j,i]))

movingimg = sitk.Image(doseimg2.shape[1], doseimg2.shape[0], sitk.sitkFloat32)
for i in range(doseimg2.shape[1]):
    for j in range(doseimg2.shape[0]):
        movingimg.SetPixel(i, j, float(doseimg2[j,i]))

initial_transform = sitk.CenteredTransformInitializer(fixedimg, 
                                                      movingimg, 
                                                      sitk.Euler2DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)


# Real registration:
registration_method = sitk.ImageRegistrationMethod()


# Similarity metric settings:
#registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=2)
registration_method.SetMetricAsMeanSquares()
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.2)

registration_method.SetInterpolator(sitk.sitkLinear)
#registration_method.SetInterpolator(sitk.sitkBSplineResamplerOrder5)


# Optimizer settings.
#registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=1000, 
#                                                    convergenceMinimumValue=1e-8, convergenceWindowSize=10)
registration_method.SetOptimizerAsRegularStepGradientDescent(learningRate=1.0, minStep=0.01, 
                                                             numberOfIterations=100, relaxationFactor=0.5)
registration_method.SetOptimizerScalesFromPhysicalShift()

registration_method.SetInitialTransform(initial_transform, inPlace=False)

final_transform = registration_method.Execute(sitk.Cast(fixedimg, sitk.sitkFloat32), 
                                              sitk.Cast(movingimg, sitk.sitkFloat32))
finalimg = sitk.Resample(movingimg, fixedimg, final_transform, sitk.sitkBSplineResamplerOrder5, 0.0, movingimg.GetPixelID())

print('   >>> rEGISTRATION dONE!')

In [8]:
# DISP REGISTRATION RESULTS:


# Diplayed img:
coefImg1 = 0.5
coefImg2 = 0.5

dispimg = coefImg2*sitk.GetArrayFromImage(finalimg) + coefImg1*doseimg1
# dispimg = sitk.GetArrayFromImage(fixedimg)

# Figure:
p1 = figure(plot_width=m_dimViewer, plot_height=int(m_dimViewer*dispimg.shape[0]/dispimg.shape[1]), 
           title='', toolbar_location="above")

p1.image(image=[dispimg], x=0, y=0, dw=dispimg.shape[1], dh=dispimg.shape[0], palette="Plasma256")

show(p1)

In [ ]:
# PLOTS PROFILES:

column = 150
line = 100


p1 = figure(plot_width=600, plot_height=400, title="Dose calculated vs real (column)", toolbar_location="above")
p2 = figure(plot_width=600, plot_height=400, title="Dose calculated vs real (line)", toolbar_location="above")

x1 = np.arange(0, len(sitk.GetArrayFromImage(finalimg)[:,column]), 1)
x2 = np.arange(0, len(doseimg1[:,column]), 1)

p1.line(x1, sitk.GetArrayFromImage(finalimg)[:,column], line_width=2, line_color='firebrick')
p1.line(x2, doseimg1[:,column], line_width=2, line_color='black', line_dash='dotted')

y1 = np.arange(0, len(sitk.GetArrayFromImage(finalimg)[line,:]), 1)
y2 = np.arange(0, len(doseimg1[line,:]), 1)

p2.line(y1, sitk.GetArrayFromImage(finalimg)[line,:], line_width=2, line_color='firebrick')
p2.line(y2, doseimg1[line,:], line_width=2, line_color='black', line_dash='dotted')


show(p1)
show(p2)

# RBGB Calibration curves:

In [ ]:
# R/B AND G/B CALCULATION:


# Reads the first image:
img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb)+m_fileExtension)

sizex = img.GetWidth()
sizey = img.GetHeight()
imgOrigin = img.GetOrigin()
imgSpacing = img.GetSpacing()

# Reads all images and does the median image:
size = (sitk.GetArrayFromImage(img).shape[0], 
        sitk.GetArrayFromImage(img).shape[1], 
        sitk.GetArrayFromImage(img).shape[2], 
        m_nbOfFiles)
imgs = np.zeros(size)

for i in range(m_nbOfFiles):
    img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb+i)+m_fileExtension)
    imgs[:,:,:,i] = sitk.GetArrayFromImage(img)
    
array = np.median(imgs, axis=3)

# array = applyStreakCorrection(array)

array_r = subSampleDataArray(array[:,:,0], 10)
array_g = subSampleDataArray(array[:,:,1], 10)
array_b = subSampleDataArray(array[:,:,2], 10)

array_rb = array_r/array_b
array_gb = array_g/array_b

doseRecalee =  sitk.GetArrayFromImage(finalimg)

In [ ]:
# CALIBRATION MODEL OF A LINE: 


# fitting functions for calibration:
def funcDose(x, a, b, c, d, e):
    return a*(x[:,0]*x[:,1])+  b*(x[:,0]**c*x[:,1]**d) + e

def funcRBGB(x, a, b, c, d):
    return a*x*x*x +  b*x*x + c*x + d


# params:
column = 160
beginLine = 80
endLine = 290
halfwidth = 2

[paramsRBGB_mean, paramsDose_mean] = createLineCalModel(array_rb, array_gb, doseRecalee, column,beginLine, endLine, 
                                              w=halfwidth, dispParameters=False, dispGraphs=True)


In [11]:
# CALIBRATION MODEL:


# fitting functions for calibration:
def funcDose(x, a, b, c, d, e):
    return a*(x[:,0]*x[:,1])+  b*(x[:,0]**c*x[:,1]**d) + e

def funcRBGB(x, a, b, c, d):
    return a*x*x*x +  b*x*x + c*x + d


# Colors for display:
colors = ['aqua', 'aquamarine', 'azure', 'bisque', 'black', 'blue', 'blueviolet', 'brown', 'burlywood', 
          'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan',
           'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey', 'darkkhaki', 'darkmagenta', 
          'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 
          'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 
          'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 
          'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'green', 'greenyellow', 'grey', 'honeydew', 'hotpink', 
          'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 
          'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgreen', 'lightgrey', 'lightpink', 
          'lightsalmon', 'lightseagreen', 'lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 
          'lightyellow', 'lime', 'limegreen', 'linen', 'magenta', 'maroon', 'mediumaquamarine', 'mediumblue', 
          'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 'mediumspringgreen', 'mediumturquoise', 
          'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 'navy', 'oldlace', 
          'olive', 'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 
          'palevioletred', 'papayawhip', 'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'red', 'rosybrown', 
          'royalblue', 'saddlebrown', 'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver', 'skyblue', 
          'slateblue', 'slategray', 'slategrey', 'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle', 'tomato', 
          'turquoise', 'violet', 'wheat']


# variables:
beginLine = 80
endLine = 290

# beginColumn = 65
# columnOffset = 10
# nbOfColumns = 95
# ColumnsHalfWidth = 5

beginColumn = 65
columnOffset = 10
nbOfColumns = 19
ColumnsHalfWidth = 4



# Creation of a list for the models:
gafModelsDict = {"RBGBcurve":[],
                "DoseCurve":[]}

for i in range(nbOfColumns):
    [paramsRBGB, paramsDose] = createLineCalModel(array_rb, array_gb, doseRecalee, beginColumn + i*columnOffset,
                                                  beginLine, endLine, w=ColumnsHalfWidth, 
                                                  dispParameters=False, dispGraphs=False)
    gafModelsDict["RBGBcurve"].append(paramsRBGB.tolist())
    gafModelsDict["DoseCurve"].append(paramsDose.tolist())

    
# Display all models: 
x = np.arange(0.7, 1.4, 0.007)  

p1 = figure(plot_width=600, plot_height=600, title="RB vs GB", 
                toolbar_location="above")

p2 = figure(plot_width=300, plot_height=int(300*array_rb.shape[0]/array_rb.shape[1]), 
                title='', toolbar_location="above")

p2.image(image=[array_rb], x=0, y=0, dw=array_rb.shape[1], dh=array_rb.shape[0], palette="Plasma256")

for i in range(len(gafModelsDict["RBGBcurve"])):
    p1.line(x, funcRBGB(x, gafModelsDict["RBGBcurve"][i][0], gafModelsDict["RBGBcurve"][i][1], 
            gafModelsDict["RBGBcurve"][i][2], gafModelsDict["RBGBcurve"][i][3]), 
            line_color=colors[i], line_width=1, line_dash='dotted')
    p2.line((beginColumn+i*columnOffset, beginColumn+i*columnOffset), (beginLine, endLine), 
            line_alpha=0.7, line_color="white")

grid = gridplot([[p1,p2]])
show(grid)

In [ ]:
# SAVING THE MODELS IN A NEW FILE:
m_jsonFileName = "calibRBGB_center_.txt"


with open(m_path+m_jsonFileName, 'w') as file:
    json.dump(gafModelsDict, file)

print(' >>> File '+m_path+m_jsonFileName+' written !')

In [ ]:
# ADDING THE LINES TO AN EXISTING MODEL:

fileToExtend = 'C:/Users/00004436/Documents/GitHub/Gafchromic_films/testRBGB/calibRBGB_center_overlappingLines.txt'
newFile = 'C:/Users/00004436/Documents/GitHub/Gafchromic_films/testRBGB/calibRBGB_center_overlappingLines_extended.txt'


# Reading file to append:
with open(fileToExtend, 'r') as file:
    gafModelsDict_toExtend = json.load(file)

    
# Appends the new dictionary at the end of the other:
for i in range(len(gafModelsDict["RBGBcurve"])):
    gafModelsDict_toExtend["RBGBcurve"].append(gafModelsDict["RBGBcurve"][i])
    gafModelsDict_toExtend["DoseCurve"].append(gafModelsDict["DoseCurve"][i])


# Write the resulting dictionary in a file:
with open(newFile, 'w') as file:
    json.dump(gafModelsDict_toExtend, file)

    
print(' >>> File '+newFile+' written !')    
